In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
#from ipywidgets import interact, interactive
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import ipywidgets as widgets
import asyncio
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import plotly
import math
import time
%matplotlib ipympl

In [3]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

In [4]:
def get_intersections(x0, y0, r0, x1, y1, r1):
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1

    d=math.sqrt((x1-x0)**2 + (y1-y0)**2)
    
    # non intersecting
    if d > r0 + r1 :
        return None
    # One circle within other
    if d < abs(r0-r1):
        return None
    # coincident circles
    if d == 0 and r0 == r1:
        return None
    else:
        a=(r0**2-r1**2+d**2)/(2*d)
        h=math.sqrt(r0**2-a**2)
        x2=x0+a*(x1-x0)/d   
        y2=y0+a*(y1-y0)/d   
        x3=x2+h*(y1-y0)/d     
        y3=y2-h*(x1-x0)/d 

        x4=x2-h*(y1-y0)/d
        y4=y2+h*(x1-x0)/d
        
        return (x3, y3, x4, y4)


In [5]:
X_limits = [-6,0]
Y_limits = [-0.06, 0.1]
R_def = 370
Fix_point = [-R_def, R_def]
t_jumps = [0,0,0,0, 0.03,0.03,0.03,0.03,0.029,0.029,0.029,0.029,0.029,0.029,0.029]
R_VTX = [2.5,5,13,18]

In [6]:
intersect3_0 = get_intersections(0,0,R_VTX[1],0,R_def,R_def-t_jumps[0])
theta3_0 = math.atan2(intersect3_0[3],intersect3_0[2])
intersect3_1 = get_intersections(0,0,R_VTX[1],0,R_def,R_def-t_jumps[4*1])


intersect2_0 = get_intersections(0,0,R_VTX[0],0,R_def,R_def-t_jumps[4*1])
theta2_0 = math.atan2(intersect2_0[3],intersect2_0[2])
intersect2_1 = get_intersections(0,0,R_VTX[0],0,R_def,R_def-t_jumps[4*2])

intersect1_0 = get_intersections(0,0,R_VTX[1],0,R_def,R_def-t_jumps[0])
theta1_0 = math.atan2(intersect3_0[3],intersect3_0[2])

intersect0_0 = get_intersections(0,0,R_VTX[1],0,R_def,R_def-t_jumps[4*1])
theta0_0 = math.atan2(intersect0_0[3],intersect0_0[2])


print(theta3_0,theta2_0)

3.134835845420086 3.1262141555405623


In [7]:
theta = np.linspace( -5.5*np.pi/4 , -2.5*np.pi/4 , 150 )
  
a = np.cos( theta )
b = np.sin( theta )


def my_track(x, x0, y0, R):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return (y0 - ( R**2 - ( x - x0 )**2 )**0.5  ) + ( R**2 - ( Fix_point[0] )**2 )**0.5 + Fix_point[1]

print(my_track(-17,0,0,29))

(346.50531975105855+368.86176272419453j)


In [8]:
anim_fig1, anim_ax1 = plt.subplots()
text_kwargs = dict(ha='center', va='center', fontsize=14, color='C0')

xsin1 = np.linspace(X_limits[0], 5, 100)
def animate(intime):
    t = int(intime/1)
    plt.cla()
    line1 = plt.plot(xsin1,my_track(xsin1,0,0,R_def-t_jumps[t]), linewidth=2)
    line2 = plt.plot(xsin1,my_track(xsin1,0,0,R_def), 'g--', linewidth=3)
    anim_ax1.plot( R_VTX[0]*a, R_VTX[0]*b , linewidth=2 )
    anim_ax1.plot( R_VTX[1]*a, R_VTX[1]*b , linewidth=2 )


    if t>0:
        theta3 = np.linspace( theta3_0-0.015 , theta3_0 + 0.015, 10 )
        a3 = np.cos( theta3 )
        b3 = np.sin( theta3 )
        anim_ax1.plot( R_VTX[1]*a3, R_VTX[1]*b3, linewidth=3 )
        theta1 = np.linspace( theta1_0-0.02 , theta1_0 + 0.02, 10 )
        a1 = np.cos( theta1 )
        b1 = np.sin( theta1 )
        anim_ax1.plot( R_VTX[0]*a1, R_VTX[0]*b1, linewidth=3 )
        if t>1:   
            theta0 = np.linspace( theta0_0-0.002 , theta0_0 + 0.002, 10 )
            a0 = np.cos( theta0 )
            b0 = np.sin( theta0 )
            anim_ax1.plot( R_VTX[1]*a0, R_VTX[1]*b0, linewidth=5 )
        if t>2:   
            anim_ax1.plot(intersect3_1[2],intersect3_1[3], 'go') 
    if t>4:
        theta2 = np.linspace( theta2_0-0.001 , theta2_0 + 0.001, 10 )
        a2 = np.cos( theta2 )
        b2 = np.sin( theta2 )
        anim_ax1.plot( R_VTX[0]*a2, R_VTX[0]*b2, linewidth=5 )  
        if t>5:
            anim_ax1.plot(intersect2_1[2],intersect2_1[3], 'go')  

    if t > 8:
        plt.text(-1., 0.059, '$DCA_T$ = 290 $\mu$m', **text_kwargs)



    anim_ax1.set_xlim(X_limits)
    anim_ax1.set_ylim(Y_limits)

    plt.xlabel("X, cm")
    plt.ylabel("Y, cm")
    plt.text(-1., 0.069, '$p_T$ = 1 GeV', **text_kwargs)
    anim_ax1.legend(['cor. DC track','def. DC track'])
    #anim_ax1.legend(line2, "cor DC track")
    #plt.xlim(- 2 * np.pi, 2 * np.pi)


matplotlib.animation.FuncAnimation(anim_fig1, animate, frames=15, interval=500, repeat_delay = 500)

/home/yoren/.local/lib/python3.10/site-packages/matplotlib/cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/yoren/.local/lib/python3.10/site-packages/matplotlib/cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


/home/yoren/.local/lib/python3.10/site-packages/matplotlib/cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/yoren/.local/lib/python3.10/site-packages/matplotlib/cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [11]:
if True:
    ani = matplotlib.animation.FuncAnimation(anim_fig1, animate, frames=15, interval=500)
    ani.save(filename="output/association.gif", writer="pillow")

/home/yoren/.local/lib/python3.10/site-packages/matplotlib/cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/yoren/.local/lib/python3.10/site-packages/matplotlib/cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [10]:
#!jupyter nbconvert --to html first.ipynb --HTMLExporter.theme=dark --no-input

import ipywidgets as widgets
import asyncio

slider = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

async def test(slider, output):
    i = 0
    while slider.value != 10:
        i = i+1
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value) + ' - ' + str(i))
        x = slider.value 
        c1= single_DCA_draw(slider.value)[0]
        if i == 1: c1.Draw()
        else : c1.Update()

output = display("tbd", display_id=True)
display(slider)

#asyncio.create_task(test(slider, output))